Step 1: Install the Required Libraries

In [ ]:
! pip install transformers torch
! pip install pdfminer.six

Step 2: Extract Text from PDF

In [11]:
from pdfminer.high_level import extract_text_to_fp
from io import StringIO

def extract_text_from_pdf(pdf_path, start_page, end_page):
    # Extracts text from a PDF file, specifying the pages for Chapter 1
    text = ""
    output = StringIO()
    with open(pdf_path, 'rb') as f:
        extract_text_to_fp(f, output, page_numbers=range(start_page-1, end_page))
        text = output.getvalue()
    return text

In [14]:
# Define the path to the PDF and the pages to extract
pdf_path = "looma_sample_book.pdf"  # Update this to the path of your PDF file
start_page = 3  # Update this to the start page of Chapter 1
end_page = 4  # Update this to the end page of Chapter 1

In [15]:
# Extract text from the specified pages
chapter_text = extract_text_from_pdf(pdf_path, start_page, end_page)
print(chapter_text)

PrefaceSchool education is the foundation for preparing the citizen who are loyal to the nation and nationality, committed to the norms and values of federal democratic republic, self-reliant and respecting the social and cultural diversity. It is also remarkable for developing a good moral character with the practical know-how of the use of ICT along with the application of scientific concept and positive thinking. It is also expected to prepare the citizens who are moral and ethical, disciplined, social and human value sensitive with the consciousness about the environmental conversation and sustainable development. Moreover, it should be helpful for developing the skills for solving the real life problems. This textbook 'Science and Technology, Grade 10' is fully aligned with the intent carried out by the National Curriculum Framework for School Education, 2076 and is developed fully in accordance with the new Secondary Level Curriculum, Grade 9-10, 2078.  This textbook is initially

Step 3: Summarize Text Using Transformers

In [ ]:
from transformers import pipeline

def summarize_text(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary_text = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary_text[0]['summary_text']

Step 4: Putting It All Together

In [ ]:
pdf_path = "looma_sample_book.pdf"  # Update this to the path of your PDF file
start_page = 1  # Update this to the start page of Chapter 1
end_page = 2  # Update this to the end page of Chapter 1

# Extract text
chapter_text = extract_text_from_pdf(pdf_path, start_page, end_page)

# Generate summary
chapter_summary = summarize_text(chapter_text)
print(chapter_summary)